In [ ]:
import speech_recognition as sr
from pydub import AudioSegment
import srt
import datetime

def transcribe_audio(audio_file, text_file):
    """
    음성 파일을 텍스트로 변환하고 단어별 타임스탬프를 생성합니다.

    Args:
        audio_file: 오디오 파일 경로
        text_file: 텍스트 파일 경로 (주어진 대사 텍스트)

    Returns:
        List: 자막 객체(srt.Subtitle) 리스트
    """

    r = sr.Recognizer()
    audio = AudioSegment.from_file(audio_file)
    
    with open(text_file, "r", encoding="utf-8") as f:
        script = f.read().split()

    # Google Web Speech API를 사용하여 음성 인식
    try:
        audio_data = sr.AudioFile(audio_file)
        with audio_data as source:
            r.adjust_for_ambient_noise(source)  # 노이즈 제거
            audio_text = r.record(source)
            result = r.recognize_google(audio_text, show_all=True, language='ko-KR')
            
        # 결과가 없는 경우 예외 처리
        if not result or 'alternative' not in result:
            raise sr.UnknownValueError("음성 인식 결과가 없거나 유효하지 않습니다.")

        words = result['alternative'][0]['transcript'].split()
    except sr.UnknownValueError:
        print("음성을 인식할 수 없습니다.")
        return []
    except sr.RequestError as e:
        print(f"Google Speech Recognition 서비스에 접근할 수 없습니다.; {e}")
        return []

    subtitles = []
    word_idx = 0
    for word_info in words:
        word = word_info['word']
        start_time = word_info['startTime'][:-1] # 's' 제거
        end_time = word_info['endTime'][:-1]  # 's' 제거
        

        start_time = datetime.timedelta(seconds=float(start_time))
        end_time = datetime.timedelta(seconds=float(end_time))

        content = ""
        while word_idx < len(script) and (content + " " + script[word_idx]).strip() == word:
            content = (content + " " + script[word_idx]).strip()
            word_idx += 1

        if content:  # 빈 content는 추가하지 않음
            subtitle = srt.Subtitle(index=len(subtitles) + 1, start=start_time, end=end_time, content=content)
            subtitles.append(subtitle)

    return subtitles

def create_srt_file(subtitles, srt_file):
    """
    srt 자막 파일을 생성합니다.

    Args:
        subtitles: 자막 객체(srt.Subtitle) 리스트
        srt_file: 생성할 srt 파일 경로
    """
    with open(srt_file, "w", encoding="utf-8") as f:
        f.write(srt.compose(subtitles))


audio_file = "/Users/1004507/work/aishorts/temp/experiment1/1.wav"  # 오디오 파일 경로를 입력하세요.
text_file = "/Users/1004507/work/aishorts/temp/experiment1/1.txt"  # 텍스트 파일 경로를 입력하세요.
srt_file = "result.srt"  # 생성할 자막 파일 경로를 입력하세요.

# 대사 텍스트 파일 생성 (이미 있다면 생략)
with open(text_file, "w", encoding="utf-8") as f:
    f.write("안녕하세요, 삼신아이 송지호입니다. 겨울철 임산부에게 추천하는 두 가지 음식 알려드릴게요! 첫 번째는 단호박이에요. 비타민 A와 섬유질이 풍부해서 태아의 성장과 임산부의 변비 예방에 좋아요. 두 번째는 고구마입니다. 따뜻한 성질을 가지고 있어서 몸을 데워주고 에너지를 보충해주죠. 이 두 가지 음식으로 건강한 겨울 보내세요! 다음에도 유익한 정보로 찾아뵐게요!")

subtitles = transcribe_audio(audio_file, text_file)
if subtitles:
    create_srt_file(subtitles, srt_file)
    print(f"자막 파일이 {srt_file}에 생성되었습니다.")

In [5]:
from pydub import AudioSegment

audio_path = '/Users/1004507/work/aishorts/temp/experiment1/1.mp3'
# MP3 파일을 WAV로 변환
audio = AudioSegment.from_mp3(audio_path)
audio.export("1.wav", format="wav")

<_io.BufferedRandom name='1.wav'>